In [5]:
import planetary_computer
from pystac_client import Client
import stackstac
import xarray as xr
import rioxarray as rio
import numpy as np
import pandas as pd
import geopandas as gpd
import os

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import rioxarray as rio
from rasterio.features import rasterize
from rasterio.enums import Resampling

from shapely import Point
import math as math

import pandas as pd
from pathlib import Path
import folium

import cv2
from tqdm import tqdm
import gc

root = Path.cwd()

In [ ]:
def map_sentinel_tiles(boundary):
    s2_grid_url = "https://unpkg.com/sentinel-2-grid/data/grid.json"
    grid_gdf = gpd.read_file(s2_grid_url)

    # find tiles that intersect with AOI
    bb = boundary.boundary.to_crs(grid_gdf.crs)
    tiles = grid_gdf[grid_gdf.intersects(bb.geometry.iloc[0])]
    exploded = tiles.explode()[0:] # separate geometry collection into single geometries

    m = folium.Map(location=(42.44,-76.21), zoom_start=5, tiles='OpenStreetMap')

    folium.GeoJson(
        exploded,
        tooltip=folium.GeoJsonTooltip(fields=["id"])
    ).add_to(m)

    folium.GeoJson(bb).add_to(m)
    
    return m

def get_sentinel_tiles(boundary):
    """Get tile ids that overlap boundary"""
    
    s2_grid_url = "https://unpkg.com/sentinel-2-grid/data/grid.json"
    grid_gdf = gpd.read_file(s2_grid_url)

    # find tiles that intersect with AOI
    bb = boundary.boundary.to_crs(grid_gdf.crs)
    tiles = grid_gdf[grid_gdf.intersects(bb.geometry.iloc[0])]
    
    return [t for t in tiles['name']]



def remove_outliers(a,min,max):  # slavi: 0-20, psri: -0.5 - 1.0
    #a = a.where(np.isfinite(a),np.nan)
        a = xr.where(np.isfinite(a),a,np.nan)
        a = a.clip(min=min,max=max)
        return a

def get_variables(data):
        blue = data.sel(band='B02')
        red = data.sel(band='B04')
        nir = data.sel(band='B8A')
        sw1 = data.sel(band='B11')
        sw2 = data.sel(band='B12')
        re2 = data.sel(band='B06')


        evi = 2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1)).expand_dims({'band':['evi']}) 
        lswi = (nir - sw1)/(nir + sw1).expand_dims({'band':['lswi']}) 
        slavi = nir/(red + sw2).expand_dims({'band':['slavi']}) 
        psri = (red - blue)/re2.expand_dims({'band':['psri']}) 

        # bright = (0.3510*blue)+(0.3813*green)+(0.3437*red)+(0.7196*nir)+(0.2396*sw1)+(0.1949*sw2).expand_dims({'band':[f'bright_{season_list[i]}']})
        # bright = self.remove_outliers(bright)

        # wet = (0.2578*blue)+(0.2305*green)+(0.0883*red)+(0.1071*nir)+(-0.7611*sw1)+(-0.5308*sw2).expand_dims({'band':[f'wet_{season_list[i]}']})
        # wet = self.remove_outliers(wet)

        # green = (-0.3599*blue)+(-0.3533*green)+(-0.4734*red)+(0.6633*nir)+(0.0087*sw1)+(-0.2856*sw2).expand_dims({'band':[f'green_{season_list[i]}']})
        # green = self.remove_outliers(green)

        evi = remove_outliers(evi,min=-1,max=1)
        lswi = remove_outliers(lswi,min=-1,max=1)
        slavi = remove_outliers(slavi,min=0,max=20)
        psri = remove_outliers(psri,min=-0.5,max=1.0)

        all_vars = xr.concat([data,evi,lswi,slavi,psri],dim='band')
        # ## mask non-forest pixels
        # self.all_variables = m3.where(m3.sel(band=f'evi_month{num_of_peak_evi_month}')>0.5,other=np.nan)
        #all_vars = xr.where(all_vars.isel(time=1,band=0)>=0.5,all_vars,np.nan)

        return all_vars




def get_gradient(im) :
        # Calculate the x and y gradients using Sobel operator
        grad_x = cv2.Sobel(im,cv2.CV_32F,1,0,ksize=3)
        grad_y = cv2.Sobel(im,cv2.CV_32F,0,1,ksize=3)
        # Combine the two gradients
        grad = cv2.addWeighted(np.absolute(grad_x), 0.5, np.absolute(grad_y), 0.5, 0)

        return grad
    

def coregister_data(data):
    # open raw sentinel data file
    #data = xr.open_dataarray(self.root / 'sentinel_data' / self.site_name / f'{self.year}_{self.site_name}.nc')

    data = data.astype('float32') # convert from float64 to save memory
    # set nans to 0
    b_sel = xr.where(~np.isnan(data), data, 0)

    # create reference image: mean of all temporal steps
    reference_image = data.mean(dim=['band','time'])
    # replace na with 0 
    reference_image = xr.where(~np.isnan(reference_image),reference_image,0)
    # convert to numpy array
    reference_image = reference_image.to_numpy()

    # define dimensions for output image
    height = b_sel.shape[2]
    width = b_sel.shape[3]
    time = b_sel.shape[0]
    band = b_sel.shape[1]

    ## Define motion model
    #warp_mode = cv2.MOTION_AFFINE
    warp_mode = cv2.MOTION_TRANSLATION

    # Set the stopping criteria for the algorithm.
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5000,  1e-10)

    # Create empty array of correct size for new aligned images
    im_aligned = np.zeros((time,band,height,width), dtype=np.float32 )

    ref_image_gradient = get_gradient(reference_image)

    #im = b_sel.to_numpy()
    # loop over time and band dimensions and apply coregistration to each band
    # calculate warp_matrix for only one band per timestamp and apply to all bands
    for i in tqdm(range(0,time)):
        im = b_sel.isel(time=i).to_numpy()  # shape: (band, y, x)
        # get just red band for calculating warp matrix
        red_band = im[2,:,:]
        red_gradient = get_gradient(red_band)
        
        # set empty warp matrix
        warp_matrix = np.eye(2, 3, dtype=np.float32)
        # calculate warp matrix based on downsampled red band and reference image
        (_, warp_matrix) = cv2.findTransformECC(ref_image_gradient, red_gradient, warp_matrix, warp_mode, criteria) 
        
        # apply transformation to each band 
        for j in range(0,band):                                
            im_aligned[i,j,:,:] = cv2.warpAffine(im[j,:,:], warp_matrix, (width,height), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
        
        del im
        gc.collect() # free memory
        
    b_align = xr.DataArray(im_aligned, 
                    coords={'time':b_sel.time,'band': b_sel.band,'y': b_sel.y,'x':b_sel.x}, 
                    dims=['time','band','y','x'])
    # reset 0 values to na
    b_align = xr.where(b_align == 0, np.nan, b_align)

    return b_align



def build_summer_timeseries(boundary, years, tiles, cf_mosaic=True):
    """Downloads Sentinel-2 data between June 15th - August 15th for each year, mosaics adjacent/overlapping tiles, masks cloud/shadows/water/snow/aerosol, applies scale factor (and offset if processing baseline < 4.0), co-registers images, calculates vegetation indices, and assembles into timeseries. 

    If cf_mosaic = False, takes the single least cloudy image per year.
    If cf_mosaic = True, creates median mosaic of all available images per year."""

    epsg = boundary.crs.to_epsg()
    bbox_4326 = tuple(boundary.to_crs(4326).total_bounds)
    bbox_utm = tuple(boundary.total_bounds)

    catalog = Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
    )


    query = {"eo:cloud_cover":{"lt":20},
            "s2:mgrs_tile": {'in':tiles}} 

    timesteps = []

    for year in years:
        items = catalog.search(
            bbox=bbox_4326,
            collections=["sentinel-2-l2a"],
            datetime=f"{year}-06-15/{year}-08-15",
            query= query #"s2:mgrs_tile": {"eq": tile_id},
        ).item_collection()
        print(f'{year}: number of images found: {len(items)}')
        if len(items) == 0:
            print(f'SKIPPING {year}')
            continue
        
        # get images with highest (i.e. most recent) processing baseline.
        # if highest baseline is less than 4.0, set apply offset to true. 
        baselines = []
        for i in items:
            baselines.append(float(i.properties.get('s2:processing_baseline')))
        highest = np.max(baselines)
        selected_items = [i for i in items if float(i.properties.get('s2:processing_baseline')) == highest]
        if highest < 4.0:
            apply_offset = True
            print(f'processing baseline: {highest} - applying offset')
        else:
            apply_offset = False
            print(f'processing baseline: {highest}')
        
        # create xarray
        stack = stackstac.stack(
            selected_items,
            epsg=epsg,
            resolution=10,
            bounds=bbox_utm,
            assets=['B02','B03','B04','B05','B06','B07','B08','B8A','B11','B12','SCL'],
            resampling=Resampling.bilinear)


        #stack = stack.chunk({'time':timechunk,'band':bandchunk,'y':None,'x':None})

        # # # crop to boundary
        stack = stack.rio.clip(geometries=boundary.geometry)

        stack  = stack.assign_coords(time=stack['time'].dt.floor('D'))

        mosaic = stack.groupby('time').median(dim='time',skipna=True) # merge images from different tiles taken on same day

        mosaic = mosaic.drop_attrs()
        mosaic = mosaic.reset_coords(drop=True)

        #mosaic = mosaic.chunk({'time':None,'band':None,'y':1024,'x':1024})

        mosaic = mosaic.astype('float32')

        scl = mosaic.sel(band='SCL')
        mask = ~scl.isin([1, 3, 6, 8, 9, 10, 11])
        #mask = ~scl.isin([1, 3, 6, 8, 9, 10, 11]).persist()

        masked = mosaic.where(mask).drop_sel(band='SCL')

        if apply_offset:
                scaled = (masked + 1000)/ 10000
        else:
                scaled = masked / 10000
        
        scaled = scaled.clip(min=0) # set minimum reflectance to 0

        #masked = masked.where(masked > 0, other=np.nan)
        # get single least cloudy image
        if not cf_mosaic:
            sample_band = scaled.isel(band=0)  
            #valid_pixel_count = da.sum(da.isfinite(sample_band.data), axis=(1, 2))
            valid_pixel_count = sample_band.count(dim=['y','x']).values
            v = valid_pixel_count.argmax() # index of timestep with most valid pixels
            scaled_lc = scaled.isel(time=v)
            timesteps.append(scaled_lc)
        else:
            timesteps.append(scaled)

    ts = xr.concat(timesteps,dim='time')

    # align
    print('aligning bands')
    aligned = coregister_data(ts)
    with_indices = get_variables(aligned)

    if cf_mosaic:  # create median mosaic of all available timesteps per year
        aggregated = with_indices.groupby('time.year').median(dim='time', skipna=True) 
        return aggregated
    else:
        return with_indices

 


In [ ]:
# version 1: take the single least cloudy image per year
# version 2: create median mosaic of all images available per year

In [11]:
b = gpd.read_file(root / 'data' / 'ordway' / 'ordway_boundary.gpkg')
boundary = b.to_crs(epsg=26917)

tiles = get_sentinel_tiles(boundary)

tsv2 = build_summer_timeseries(boundary,years=[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025], tiles=tiles,cf_mosaic=True)


2016: number of images found: 3
processing baseline: 2.12 - applying offset
2017: number of images found: 1
processing baseline: 2.12 - applying offset
2018: number of images found: 4
processing baseline: 2.12 - applying offset
2019: number of images found: 4
processing baseline: 2.12 - applying offset
2020: number of images found: 6
processing baseline: 2.12 - applying offset
2021: number of images found: 3
processing baseline: 3.0 - applying offset
2022: number of images found: 6
processing baseline: 5.1
2023: number of images found: 4
processing baseline: 5.1
2024: number of images found: 0
SKIPPING 2024
2025: number of images found: 6
processing baseline: 5.11


100%|██████████| 22/22 [05:55<00:00, 16.14s/it]
